In [5]:
import streamlit as st
import os
import ollama
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.embeddings import OllamaEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI


In [1]:
import streamlit as st
import replicate
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables (for Replicate API key)
load_dotenv(find_dotenv())

# List of available Replicate models
replicate_model_names = [
    'meta/meta-llama-3-8b',
    'google-deepmind/gemma2-9b-it',
    'microsoft/phi-3-medium-4k-instruct',
    'mistralai/mixtral-8x7b-instruct-v0.1'
]

# Streamlit app
st.title("Replicate Model Stream Interface")

# Sidebar for selecting model and input parameters
with st.sidebar:
    st.header("Model Selection & Settings")

    # Dropdown for selecting Replicate model
    selected_model = st.selectbox("Select Replicate Model:", replicate_model_names)

    # Text area for custom prompt input
    prompt_input = st.text_area("Enter the prompt:", value="Story title: 3 llamas go for a walk\nSummary: The 3 llamas crossed a bridge and something unexpected happened\n\nOnce upon a time")

    # Optional settings for the model
    temperature = st.slider("Temperature:", min_value=0.0, max_value=1.0, value=0.6)
    top_p = st.slider("Top P:", min_value=0.0, max_value=1.0, value=0.9)
    presence_penalty = st.slider("Presence Penalty:", min_value=0.0, max_value=2.0, value=1.15)

# Prepare the input for the Replicate model
input = {
    "prompt": prompt_input,
    "min_tokens": 0,
    "temperature": temperature,
    "top_p": top_p,
    "presence_penalty": presence_penalty
}

# Button to trigger the Replicate model
if st.button("Generate Response"):
    st.write(f"Using model: {selected_model}")
    st.write("Response:")

    # Streaming response from replicate using the selected model
    response = ""
    for event in replicate.stream(selected_model, input=input):
        response += event['text']
        st.write(event['text'], end="")  # Show real-time output


2024-09-19 02:03:43.872 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.096 
  command:

    streamlit run c:\Users\bhuva\anaconda3\envs\RAG\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-19 02:03:44.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-19 02:03:44.111 Thread 'MainTh

In [16]:
!pip show ollama

Name: ollama
Version: 0.3.3
Summary: The official Python client for Ollama.
Home-page: https://ollama.ai
Author: Ollama
Author-email: hello@ollama.com
License: MIT
Location: C:\Users\bhuva\anaconda3\envs\RAG\Lib\site-packages
Requires: httpx
Required-by: 


In [18]:
#@title **Nomic-embeddings**
!ollama pull nomic-embed-text
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB 

In [30]:
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb..

In [31]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
llama3.1:8b            	42182419e950	4.7 GB	Less than a second ago	
nomic-embed-text:latest	0a109f422b47	274 MB	2 hours ago           	


In [9]:
# Paths
from langchain_community.document_loaders import PDFPlumberLoader

pdf_directory = 'data'
vector_database_path = "vector_database"
audio_path = "audio_files"

# Create directories if they don't exist
os.makedirs(pdf_directory, exist_ok=True)
os.makedirs(vector_database_path, exist_ok=True)
os.makedirs(audio_path, exist_ok=True)

# Function to process PDFs
def process_pdfs(pdf_files):
    text_chunks = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        loader = PDFPlumberLoader(pdf_path)
        pages = loader.load_and_split()
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
        for page in pages:
            chunks = text_splitter.split_text(page.page_content)
            text_chunks.append(chunks)
    return text_chunks

In [10]:
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]
text_chunks = process_pdfs(pdf_files)
text_chunks

[['Attention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain\nllion@google.com aidan@cs.toronto.edu lukaszkaiser@google.com\nIlliaPolosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThedominantsequencetransductionmodelsarebasedoncomplexrecurrentor\nconvolutionalneuralnetworksthatincludeanencoderandadecoder. Thebest\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbasedsolelyonattentionmechanisms,dispensingwithrecurrenceandconvolutions\nentirely. Experiments on two machine translation tasks show these models to\nbesuperiorinqualitywhilebeingmoreparallelizableandrequiringsignificantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014

In [11]:
# Function to generate embeddings
def generate_embeddings(text_chunks, model_name='nomic-embed-text'):
    embeddings = []
    for chunk_list in text_chunks:
        for chunk in chunk_list:
            embedding = ollama.embeddings(model_name, prompt=chunk)
            embeddings.append(embedding)
    return embeddings

In [20]:
generate_embeddings = generate_embeddings(text_chunks)

In [21]:
generate_embeddings

[{'embedding': [1.031677007675171,
   0.3003525137901306,
   -2.223639726638794,
   -1.073697805404663,
   0.2509891092777252,
   -0.3031254708766937,
   1.0796147584915161,
   0.4718725085258484,
   -0.33908388018608093,
   0.21013779938220978,
   -0.6789125204086304,
   0.30559542775154114,
   1.5714726448059082,
   1.0435031652450562,
   0.2008773237466812,
   0.05055053159594536,
   -0.17195281386375427,
   -0.18923397362232208,
   -0.3918778896331787,
   -0.2568788528442383,
   -0.6435627341270447,
   -0.2990460693836212,
   0.28983786702156067,
   -0.0647464394569397,
   0.18510910868644714,
   0.8696515560150146,
   -0.6828269958496094,
   0.19238077104091644,
   -0.9987956881523132,
   -0.1796475648880005,
   0.7975765466690063,
   0.3021114468574524,
   -0.01649666018784046,
   0.005203898996114731,
   -0.2990354895591736,
   0.12876111268997192,
   1.3576328754425049,
   -0.2893439531326294,
   -0.16229656338691711,
   0.5866652131080627,
   0.515150249004364,
   -0.021785318

In [22]:
# Function to setup vector database
def setup_vector_database(text_chunks):
    documents = [Document(page_content=chunk) for chunk_list in text_chunks for chunk in chunk_list]
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding= OllamaEmbeddings(model='nomic-embed-text', show_progress=False),
        persist_directory=vector_database_path,
        collection_name='Local-Rag'
    )
    vector_db.persist()
    return vector_db

In [32]:
vector_db = setup_vector_database(text_chunks)

In [33]:
# Function to query the model
def query_model(vector_db, question):
    local_model = "llama3.1:8b"
    llm = ChatOllama(model=local_model)

    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate five
        different versions of the given user question to retrieve relevant documents from
        a vector database. By generating multiple perspectives on the user question, your
        goal is to help the user overcome some of the limitations of the distance-based
        similarity search. Provide these alternative questions separated by newlines.
        Original question: {question}""",
    )

    retriever = MultiQueryRetriever.from_llm(
        retriever=vector_db.as_retriever(),
        llm=llm,
        prompt=QUERY_PROMPT
    )

    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}  # Pass through the question
        | prompt  # Apply the prompt with the retriever context
        | llm  # Use the LLM to generate the final response
        | StrOutputParser()  # Parse the output as a string
    )

    response = chain.invoke(question)
    return response

In [34]:
# Question input
questions = "Describe the Self Attention Mechanism"
response = query_model(vector_db, questions)
display(Markdown(f"**Answer:**\n{response}"))

"The Self-Attention Mechanism in the Transformer Model is a key component that enables the model to weigh and combine information from different positions in the input sequence. This mechanism allows the model to attend to all positions simultaneously, rather than relying on sequential processing as in recurrent neural networks (RNNs).\n\nHere are the details of how the Self-Attention Mechanism works:\n\n1.  **Query**: The first step is to create a query vector for each position in the input sequence. This is done by multiplying the input embeddings with a learned weight matrix.\n2.  **Key**: The next step is to create key vectors, which are also obtained by multiplying the input embeddings with a different set of learned weights.\n3.  **Value**: Similar to the query and key vectors, value vectors are created by multiplying the input embeddings with another set of learned weights.\n4.  **Attention Weight Calculation**: For each position in the sequence, an attention weight is calculate